# TĂNG CƯỜNG DỮ LIỆU

# **Thông tin của tác giả, ngày cập nhật**

<hr>

**Thành viên nhóm**:
- **Trần Đình Khánh Đăng - 22520195**
- **Tăng Nhất - 22521027**
- **Lê Minh Nhựt - 22521060**

**Ngày cập nhật**: 22/01/2025

## Import thư viện cần thiết

In [ ]:
import os
import random
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from tqdm.notebook import tqdm
from tensorflow.keras.applications import (MobileNet, MobileNetV2, MobileNetV3Small, MobileNetV3Large, 
                                           ResNet50, ResNet101, ResNet152,
                                           VGG16, VGG19,
                                           EfficientNetB0, EfficientNetB1, EfficientNetB7,
                                           InceptionV3, Xception)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input

from skimage.io import imread
from skimage.color import rgb2gray
from skimage.metrics import structural_similarity as ssim

## Khởi tạo đường dẫn

In [ ]:
cropped_base_dir= '/kaggle/input/cs114-cropped-full-dataset/dataset'
cropped_dataset_name = 'cropped_CarDataset.csv'
cropped_file_name_cars = 'cropped_CarDataset-1.csv'
cropped_file_name_categories = 'cropped_CarDataset-2.csv'
cropped_extracted_features_file_name='cropped_extracted_features.npz'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, base_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.base_dir = base_dir
        self.transform = transform

        # Duyệt thư mục lớn để tạo từ điển map từ folder đến ảnh
        self.image_paths = self._find_image_paths()

    def _find_image_paths(self):
        # Tạo từ điển chứa tên folder và đường dẫn đầy đủ tới ảnh trong folder đó
        image_paths = {}
        
        # Use tqdm to show progress while iterating through folders
        for folder_name in tqdm(self.data_frame.iloc[:, 0].values, desc="Scanning folders"):
            folder_path = os.path.join(self.base_dir, folder_name)
            if os.path.isdir(folder_path):  # Kiểm tra nếu folder tồn tại
                for img_file in os.listdir(folder_path):
                    # Thêm ảnh vào từ điển với key là tên folder và value là đường dẫn ảnh
                    img_path = os.path.join(folder_path, img_file)
                    if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                        image_paths[img_file] = img_path
        return image_paths

    def __len__(self):
        return len(self.data_frame)  # Corrected from self.data_frames to self.data_frame

    def __getitem__(self, idx):
        # Lấy tên folder từ file CSV
        folder_name = self.data_frame.iloc[idx, 0]

        # Lấy class tương ứng từ file CSV
        label = int(self.data_frame.iloc[idx, 1])

        # Lấy ảnh đầu tiên từ folder tương ứng
        img_path = None
        img_path = os.path.join(self.base_dir, folder_name)  # Corrected from base_dir to self.base_dir

        if img_path is None:
            raise FileNotFoundError(f"Không tìm thấy ảnh trong thư mục {folder_name}")

        # Mở ảnh bằng PIL
        image = Image.open(img_path).convert("RGB")

        # Áp dụng các phép biến đổi ảnh nếu có
        if self.transform:
            image = self.transform(image)

        return image, label, img_path

In [ ]:
csv_file = "/kaggle/input/cs114-extracted-features/dropdup_extracted_features.csv"

# Phép biến đổi ảnh (resize, chuyển thành tensor, normalize)
img_size = 640

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),   # Thay đổi kích thước ảnh
    transforms.ToTensor(),         # Chuyển thành tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize ảnh
])

# Tạo dataset từ file CSV và thư mục ảnh
train_dataset = CustomImageDataset(csv_file=csv_file,
                                   base_dir=cropped_base_dir,
                                   transform=transform)

# Tạo DataLoader để load dữ liệu
dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)

# Hàm hiển thị ảnh với khả năng điều chỉnh kích thước figure
def imshow(img, figsize=(30, 30)):  # figsize là kích thước (width, height) của figure
    # Đảo ngược chuẩn hóa ảnh
    img = img / 2 + 0.5  # Đảo ngược chuẩn hóa (vì đã chuẩn hóa với mean=0.5, std=0.5)
    npimg = img.numpy()  # Chuyển tensor thành numpy
    plt.figure(figsize=figsize)  # Thiết lập kích thước figure
    plt.imshow(np.transpose(npimg, (1, 2, 0)))  # Đảo chiều từ (C, H, W) -> (H, W, C)
    plt.grid(False)
    plt.show()

# Kiểm tra dữ liệu bằng cách in ra 1 batch và hiển thị ảnh
for images, labels, _ in tqdm(dataloader, total=dataloader.__len__(), desc="Processing batches"):  # Thêm tqdm ở đây
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")

    # Hiển thị batch ảnh đầu tiên (với kích thước lớn hơn)
    imshow(torchvision.utils.make_grid(images), figsize=(12, 12))  # Kích thước lớn hơn (width=12, height=12)

    break  # Hiển thị 1 batch thôi

In [ ]:
def pytorch_to_numpy(image_tensor):
    return image_tensor.cpu().permute(1, 2, 0).numpy()  # Chuyển từ (C, H, W) thành (H, W, C)

# Hàm hiển thị ảnh
def plot_images(original_img, augmented_imgs, n_samples=5, figsize=(15, 5)):
    plt.figure(figsize=figsize)

    # Vẽ ảnh gốc ở vị trí đầu tiên
    plt.subplot(1, n_samples + 1, 1)
    plt.imshow(original_img)
    plt.axis('off')
    plt.title("Original Image")

    # Vẽ các ảnh augmented ở các vị trí còn lại
    for i in range(n_samples):
        plt.subplot(1, n_samples + 1, i + 2)
        plt.imshow(augmented_imgs[i])
        plt.axis('off')
        plt.title(f"Augmented {i+1}")

    plt.show()

# Hàm áp dụng data augmentation bằng PyTorch
def apply_augmentations_with_pytorch(dataloader,
                                     n_samples=7, 
                                     output_dir="/kaggle/working/augmented_images", 
                                     plot_images=True,
                                     verbose=True):
    # Tạo thư mục đầu ra nếu chưa tồn tại
    os.makedirs(output_dir, exist_ok=True)

    # Lấy một ảnh mẫu để có kích thước ảnh
    for images, _, img_paths in dataloader:  # Thêm img_paths vào vòng lặp
        img_size = images[0].shape[1]  # Lấy chiều cao hoặc chiều rộng vì ảnh là vuông
        break

    # Khởi tạo các phép biến đổi sử dụng torchvision.transforms
    transform = transforms.Compose([
        transforms.RandomRotation(30),         # Quay ngẫu nhiên ảnh trong phạm vi 30 độ
        transforms.RandomHorizontalFlip(),     # Lật ngang ngẫu nhiên
        transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),  # Cắt và thay đổi kích thước ngẫu nhiên
        transforms.ColorJitter(
            brightness=random.uniform(0, 0.1),
            contrast=random.uniform(0.5, 0.7),
            saturation=random.uniform(0.5, 0.7),
            hue=random.uniform(0, 0.2)
        )  # Điều chỉnh thông số ColorJitter hợp lý hơn
    ])

    # Lặp qua dữ liệu trong dataloader với tqdm
    for batch_idx, (images, _, img_paths) in enumerate(tqdm(dataloader, total=dataloader.__len__(), desc="Processing batches")):
        # Chuyển mỗi batch ảnh từ tensor sang numpy array
        for image_tensor, img_path in zip(images, img_paths):  # Lặp qua từng ảnh và đường dẫn
            # In ra đường dẫn ảnh
            print(f"Image Path: {img_path}") if verbose else None
            image_tensor = image_tensor.to(device)

            # Ảnh gốc
            original_image_np = pytorch_to_numpy(image_tensor)
            original_image_np = (original_image_np * 0.5 + 0.5).clip(0, 1)  # Đảo ngược chuẩn hóa

            # Áp dụng augmentations
            augmented_imgs = []
            for aug_idx in range(n_samples):
                # Áp dụng phép biến đổi cho mỗi ảnh
                augmented_image_tensor = transform(image_tensor * 0.5 + 0.5)
                augmented_image_np = pytorch_to_numpy(augmented_image_tensor)
                augmented_image_np = (augmented_image_np).clip(0, 1)  # Đảm bảo ảnh sau augment nằm trong khoảng [0, 1]
                augmented_imgs.append(augmented_image_np)

                # Lấy tên folder từ đường dẫn ảnh
                folder_name = img_path.split('/')[-2]  # Lấy tên folder từ đường dẫn (ví dụ: "Honda")
                folder_output_dir = os.path.join(output_dir, folder_name)  # Thư mục đầu ra cho folder này
                os.makedirs(folder_output_dir, exist_ok=True)  # Tạo thư mục nếu chưa tồn tại

                # Lưu ảnh đã augment
                img_name = os.path.basename(img_path)  # Lấy tên file từ đường dẫn
                img_name_without_ext = os.path.splitext(img_name)[0]  # Bỏ phần mở rộng
                output_path = os.path.join(folder_output_dir, f"{img_name_without_ext}_augmented_{aug_idx + 1}.jpg")
                save_image(augmented_image_tensor.cpu(), output_path)  # Lưu ảnh (chuyển lại về CPU trước khi lưu)

            # Plot ảnh gốc và các ảnh augmented nếu plot_images=True
            if plot_images:
                plot_images(original_image_np, augmented_imgs, n_samples=n_samples)

In [ ]:
apply_augmentations_with_pytorch(dataloader,
                                 n_samples=5,
                                 plot_images=False,
                                 verbose=False)